In [1]:
import pandas as pd

def assign_incrementality_weights(values, factors):
    # Step 1: create df to store initial values and further columns
    results = pd.DataFrame({'channel': [channel for channel in values.keys()], 'value': [value for value in values.values()]})

    # Step 2: calculate the total value volume
    total_value = 0
    for channel, value in values.items():
        total_value += value

    # Step 3: calculate the share of non-multiplied keys
    # Share for multiplied keys is irrelevant but an improvement can be made to null that
    non_multiplied_value = 0
    for channel, value in values.items():
        if channel not in factors.keys():
            non_multiplied_value += value
    if non_multiplied_value > 0:
        for channel, value in values.items():
            results['share'] = results['value'] / non_multiplied_value
    else:
        results["share"] = 0

    # Step 4: iterate through the values dictionary and multiply the values by the corresponding factor
    for i, row in results.iterrows():
        if row['channel'] in factors.keys():
            factor = factors.get(channel) # get the multiplying factor for the channel
            new_value = value * factor
            results.at[i, 'new_value'] = new_value
        else:
            results.at[i, 'new_value'] = results.at[i, 'value']

    # Step 5: calculate the surplus
    surplus = (results['new_value'].sum()) - total_value

    # Step 6: calculate the max possible subtraction for non-multiplied countries
    for i, row in results.iterrows():
        if row['channel'] not in factors.keys():
            abs_subtraction = row['share'] * surplus
            results.at[i, 'abs_subtraction'] = abs_subtraction
            max_subtraction = min(abs(abs_subtraction), row['value'])
            results.at[i, 'max_subtraction'] = max_subtraction
        else:
            results.at[i, 'max_subtraction'] = 0.0
            results.at[i, 'abs_subtraction'] = 0.0

    # Step 7: calculate the remaining surplus if the possible substraction is lower than reduciton required
    remaining_surplus = surplus - results['max_subtraction'].sum()

    # Step 8: distribute the remaining surplus proportionally among the multiplied countries
    if remaining_surplus > 0:
        for i, row in results.iterrows():
            if row['channel'] in factors.keys():
                proportion = row['value'] / results.loc[results['channel'].isin(factors.keys()), 'value'].sum()
                subtraction = proportion * remaining_surplus
                results.at[i, 'new_value'] -= subtraction

    # Step 9: calculate the final share of each channel
    for i, row in results.iterrows():
        if row['channel'] not in factors.keys():
            if results.at[i, 'max_subtraction'] == 0:
                results.at[i, 'weights'] = 1
            else:
                results.at[i, 'weights'] = (results.at[i, 'new_value'] - results.at[i, 'max_subtraction']) / results.at[i, 'value']
        else:
            if remaining_surplus > 0:
                results.at[i, 'weights'] = (results.at[i, 'new_value'] - results.at[i, 'max_subtraction']) / results.at[i, 'value']
            else:
                results.at[i, 'weights'] = factors.get(row['channel'])
    
    results["final_volume"] = results['value'] * results["weights"]
    
    # Step 10: return results
    return results[["channel", "value", "final_volume", "weights"]]

In [2]:
starting_case = {'Google search': 60, 'Direct' : 30, 'GAC': 5, 'Meta': 5}
one_zero_case = {'Google search': 5, 'Direct' : 30, 'GAC': 5, 'Meta': 5}
two_zeros_case = {'Google search': 0, 'Direct' : 0, 'GAC': 5, 'Meta': 5}
factors = {'GAC': 2, 'Meta': 2}

In [3]:
assign_incrementality_weights(starting_case, factors)

,channel,value,final_volume,weights
0,Google search,60,53.333333,0.888889
1,Direct,30,26.666667,0.888889
2,GAC,5,10.000000,2.000000
3,Meta,5,10.000000,2.000000


In [4]:
assign_incrementality_weights(one_zero_case, factors)

,channel,value,final_volume,weights
0,Google search,5,3.571429,0.714286
1,Direct,30,21.428571,0.714286
2,GAC,5,10.000000,2.000000
3,Meta,5,10.000000,2.000000


In [5]:
assign_incrementality_weights(two_zeros_case, factors)

,channel,value,final_volume,weights
0,Google search,0,0.0,1.0
1,Direct,0,0.0,1.0
2,GAC,5,5.0,1.0
3,Meta,5,5.0,1.0
